In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
# load the data
df = pd.read_csv("../datasets/BTC-USD.csv")
# define features and target
look_back = 20
feature_df = df.drop(columns="Date", axis=1)
features = []
targets = []
for i in range(look_back, len(df) - 1):
    features.append(feature_df.values[i-look_back:i])
    price_delta = df["Close"].values[i+1] - df["Close"].values[i]
    targets.append(0 if price_delta < 0 else 1)
# convert to numpy arrays for use with XGBoost
features = np.array(features)
targets = np.array(targets)
features_flattened = features.reshape(features.shape[0], -1)
# create regression matrices
d_train = xgb.DMatrix(features_flattened, label=targets)

In [3]:
training_params = {
    "objective": "reg:logistic",
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 1,
    "colsample_bytree": 0.9
}
model = xgb.train(
    params=training_params,
    dtrain=d_train,
    num_boost_round=200
)

In [4]:
model.save_model("../weights/xgb_next_day_trend.json")